In [1]:
import spacy

nlp = spacy.load("en_core_web_lg")

In [2]:
import pandas as pd

query = "Query_AI-magazines-mediacloud-2021-24"

CSV_FILE = query + ".csv"
FEATHER_FILE = query + ".feather"

df = pd.read_feather(FEATHER_FILE)

df.sample(3)

,id,indexed_date,language,media_name,media_url,publish_date,title,url,year,text,text_downloaded,GPE,IND,ORG
2482,ce29e4fc0d14e3461f9277d8128d4441d0b2673d81bdab...,2024-09-02 21:57:43.585826,en,wired.com,wired.com,2021-04-28,Ford's Ever-Smarter Robots Are Speeding Up the...,https://www.wired.com/story/fords-smarter-robo...,2021,Application\nHuman-computer interaction\nEnd U...,True,"[Livonia, Michigan, Livonia, Livonia, Livonia]","[Henry Ford, Lon Van Geloven, Van Geloven, Elo...","[Ford, Ford Transmission Plant, Ford, Symbio R..."
940,b0eb0bad093d1d990e2d88f1930df1e173db8835e9a8ba...,2024-02-20 09:42:19.028505,en,wired.com,wired.com,2023-09-27,Hollywood Writers Reached an AI Deal That Will...,https://www.wired.com/story/us-writers-strike-...,2023,,False,[],[],[]
948,6cbd8fea9e486cbb3fbf8432b27381acac7feada126144...,2024-02-19 21:31:20.790098,en,wired.com,wired.com,2023-09-26,What If the Robots Were Very Nice While They T...,https://www.wired.com/story/ai-diplomacy-robots,2023,The Morrissey had the right melodrama in his l...,True,"[Manhattan, Morrissey, Morrissey, Morrissey, S...","[Morrissey, Smiths, Smiths, Morrissey, Moz, Sm...","[Diplomacy, the North American Diplomacy Feder..."


In [3]:
df['text'] = df['text'].fillna('')

In [4]:
from tqdm.notebook import tqdm

GPE = [""] * len(df)
IND = [""] * len(df)
ORG = [""] * len(df)

# Browse the data ...
for index, row in tqdm(df.iterrows(), total=len(df)):
    text = row.text
    doc = nlp(text)

    entities = {
        "GPE": [],
        "IND": [],
        "ORG": [],
    }

    for ent in (e for e in doc.ents if e.label_ == "GPE"):
        entities["GPE"].append(ent.text)

    for ent in (e for e in doc.ents if e.label_ == "PERSON"):
        entities["IND"].append(ent.text)

    for ent in (e for e in doc.ents if e.label_ == "ORG"):
        entities["ORG"].append(ent.text)

    GPE[index] = entities["GPE"]
    IND[index] = entities["IND"]
    ORG[index] = entities["ORG"]

  0%|          | 0/2633 [00:00<?, ?it/s]

In [6]:
df['GPE'] = GPE
df['IND'] = IND
df['ORG'] = ORG

df.to_feather(FEATHER_FILE)